In [ ]:
# CUDA 환경 초기화 및 메모리 정리
import torch
import gc
import os
import json
import os
from rfdetr import RFDETRBase,RFDETRSmall
# CUDA 캐시 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.current_device()}")
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 파이썬 메모리 정리o'
gc.collect()
# CUDA 디버깅 환경 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print("CUDA 환경 초기화 완료")
device='cuda:0'

In [ ]:
# 데이터셋 검증 먼저 수행


# 데이터 경로 설정
data_path = '../../data/coco_IGNITE'
result_path = '../../results/TPS_RFDETR'
model_path = '../../model/TPS_RFDETR'

os.makedirs(result_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

# COCO 데이터셋 유효성 검사
def validate_coco_dataset(data_path):
    try:
        # train 데이터 확인
        with open(os.path.join(data_path, 'train/_annotations.coco.json'), 'r') as f:
            train_data = json.load(f)
        
        print(f"✓ Train 데이터:")
        print(f"  - 이미지 수: {len(train_data['images'])}")
        print(f"  - 어노테이션 수: {len(train_data['annotations'])}")
        print(f"  - 카테고리 수: {len(train_data['categories'])}")
        
        # 카테고리 정보 출력
        for cat in train_data['categories']:
            cat_count = len([ann for ann in train_data['annotations'] if ann['category_id'] == cat['id']])
            print(f"    - {cat['name']} (ID: {cat['id']}): {cat_count}개")
        
        # 유효한 category_id 범위 확인
        valid_cat_ids = [cat['id'] for cat in train_data['categories']]
        ann_cat_ids = [ann['category_id'] for ann in train_data['annotations']]
        invalid_ids = [cid for cid in ann_cat_ids if cid not in valid_cat_ids]
        
        if invalid_ids:
            print(f"⚠️ 잘못된 category_id 발견: {set(invalid_ids)}")
            return False
        else:
            print("✓ 모든 category_id가 유효합니다")
            return True
            
    except Exception as e:
        print(f"❌ 데이터셋 검증 실패: {e}")
        return False

# 데이터셋 검증 실행
if validate_coco_dataset(data_path):
    print("\n✓ 데이터셋 검증 완료. 모델 초기화를 진행합니다...")
    
    # 모델 초기화 (클래스 수 명시적 설정)
    model = RFDETRSmall(
        num_classes=3,  # pd-l1 negative(1), pd-l1 positive(2), non-tumor(3)
        device='cuda'
    )
    
    print("✓ 모델 초기화 완료")
else:
    print("❌ 데이터셋에 문제가 있습니다. COCO 데이터 생성을 다시 확인해주세요.")


In [ ]:
model.train(
    dataset_dir=data_path,
    epochs=1000,
    batch_size=4,
    grad_accum_steps=1,
    lr=1e-4,
    output_dir=model_path,
)